In [ ]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score

import torch
from torchvision import transforms
import albumentations as albu
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision import models as models
from torch.utils.data import Dataset, DataLoader
import torchvision
import cv2

device = ('cuda:0' if torch.cuda.is_available() else 'cpu')  # включаем поддержку GPU

%matplotlib inline
device

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from PIL import Image
from os import listdir
from os.path import isfile, join

In [ ]:
train_dir = 'avia-train/avia-train/'
test_dir = 'avia-test/avia-test/'

images_train_filenames =  [f for f in listdir(train_dir) if isfile(join(train_dir, f))]
images_test_filenames =  [f for f in listdir(test_dir) if isfile(join(test_dir, f))]

In [ ]:
train_cl_nm = pd.read_csv('train.csv')
test_nm = pd.read_csv('test.csv')

In [ ]:
train_cl_nm.head()

In [ ]:
def convert_image_to_array(img, image_size):
    try:
        return cv2.resize(img, (image_size, image_size), interpolation = cv2.INTER_AREA)
    except BaseException as e:
        print('Error!')
        print(e)
        plt.imshow(np.array(img) / 255)

In [ ]:
image_size = 128

In [ ]:
X_train, y_train = [], []

In [ ]:
for index, row in tqdm(train_cl_nm.iterrows()):
    im = cv2.imread(train_dir + row['filename'] + '.png')
#     im_array = convert_image_to_array(im, image_size=image_size)
    X_train.append(im)
    y_train.append(row['sign'])

In [ ]:
X = np.array(X_train)
y = np.array(y_train).reshape(-1,1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 256, test_size = 0.3)

In [ ]:
def train_augmentation():
    transform = [
        albu.Resize(height=image_size, width=image_size, p=1),
        albu.OneOf(
            [
                albu.HorizontalFlip(p = 0.7),
                albu.VerticalFlip(p = 0.2),
                albu.RandomRotate90(p = 0.1),
            ],
            p = 0.7
        ),
        albu.OneOf(
            [
                albu.GaussNoise(p=0.5),
                albu.RandomGamma(p=0.2),
                albu.RandomBrightnessContrast(p=0.3),
            ],
            p = 0.4
        ),
        albu.ShiftScaleRotate(
            shift_limit=0.002, scale_limit=0.2, rotate_limit=180,
            p=0.1
        )
    ]
    return albu.Compose(transform, p = 1)

In [ ]:
def valid_augmentation():
    transform = [
        albu.Resize(height=image_size, width=image_size, p=1),
    ]
    return albu.Compose(transform)

In [ ]:
class Airplane(Dataset):
    def __init__ (self, data, labels, augmentation = None):
        self.X = data
        self.y = labels
        self.aug = augmentation
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        img = self.X[i]
        if self.aug:
            sample = np.array(self.aug(image=img))
        label = self.y[i]
        return torch.tensor(np.moveaxis(sample.reshape(-1)[0]['image'], -1, 0), dtype=torch.float), torch.tensor(label, dtype=torch.float)

In [ ]:
net = models.resnet18(pretrained=True).to(device)  # загружаем предобученную на ImageNet resnet152 сразу на GPU
fc_inputs = net.fc.in_features

net.fc = nn.Sequential(
    nn.Linear(fc_inputs, 1)
).to(device)
net.aux_logits=False

for param in net.parameters():
    param.requires_grad = False
    
for param in net.fc.parameters():  # включаем последний слой (классификатор)
    param.requires_grad = True

for param in net.layer1.parameters():
    param.requires_grad = True
    
for param in net.layer2.parameters():
    param.requires_grad = True

for param in net.layer3.parameters():
    param.requires_grad = True
    
for param in net.layer4.parameters():
    param.requires_grad = True

# Stratified kfold

In [ ]:
def train_model(model, device, loss_fn, optimizer, train_loader, val_loader, num_epoch, n_fold):
    train_losses = []
    test_losses = []
    acc = []

    for i in range(num_epoch):
        epoch_train_losses = []
        model.train(True)
        for X_train, y_train in tqdm(train_loader):
            # Посчитаем предсказание и лосс
            X_train = X_train.to(device)
            y_train = y_train.to(device)
            y_pred = model(X_train)
            loss = loss_fn(y_pred, y_train)
            del y_pred

            # зануляем градиент
            optimizer.zero_grad()

            # backward
            loss.backward()

            # ОБНОВЛЯЕМ веса
            optimizer.step()

            # Запишем число (не тензор) в наши батчевые лоссы
            epoch_train_losses.append(loss.item())   
                    
        train_losses.append(np.mean(epoch_train_losses))
        
        # Теперь посчитаем лосс на вал
        with torch.no_grad():
            model.eval()
            epoch_test_losses = []
            epoch_acc = []
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_pred = model(X_val)
                loss = loss_fn(y_pred, y_val)
            
                epoch_test_losses.append(loss.item())
                y_pred = y_pred.sigmoid().detach().cpu().numpy()
                y_pred = (y_pred>=0.5).astype(int)
                epoch_acc.append(accuracy_score(y_val.cpu(), y_pred))
                del y_pred

            test_losses.append(np.mean(epoch_test_losses))
            acc.append(np.mean(epoch_acc))
            
            torch.save(model.state_dict(), f'epoch_{i}_fold_{n_fold}.pth')  # сохраняем веса эпох

            print(
                'Train loss =', train_losses[-1],
                'Val loss =', test_losses[-1],
                'Val accuracy score =', acc[-1]
            )
        if i == 5:
            for g in optimizer.param_groups:
                g['lr'] = g['lr']*0.8
            
    return train_losses, test_losses, acc

In [ ]:
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for inputs in dataloader:

            outputs = model(inputs.to(device))
            y_pred = outputs.sigmoid().detach().cpu().numpy()

            preds.append(y_pred)

    preds = np.concatenate(preds)
    
    return preds

In [ ]:
class Airplane_test(Dataset):
    def __init__ (self, data, augmentation = None):
        self.X = data
        self.aug = augmentation
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        img = self.X[i]
        if self.aug:
            sample = np.array(self.aug(image=img))
        return torch.tensor(np.moveaxis(sample.reshape(-1)[0]['image'], -1, 0), dtype=torch.float)

In [ ]:
test = []
for index, row in tqdm(test_nm.iterrows()):
    im = cv2.imread(test_dir + row['filename'] + '.png')
#     im_array = convert_image_to_array(im, image_size=image_size)
    test.append(im)
# for i in range(len(test)):
#     if test[i].shape != (image_size, image_size, 3):
#         test[i] = test[i][:,:,:3]
X_test = np.array(test)
inference_data = Airplane_test(X_test, augmentation = valid_augmentation())

# dataloaders - с помощью нашего класса датасета сэмплируют данные в батчи
BATCH_SIZE = 128
inference_dataloader = DataLoader(inference_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Обучение
#
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5)
n_fold = 0
y_preds = np.zeros(1000)
best_epochs = []

for train_index, test_index in kfold.split(X, y):
    print("Fold", n_fold)
    X_train, y_train = X[train_index], y[train_index]
    X_valid, y_valid = X[test_index], y[test_index]
    
    
    train_data = Airplane(X_train, y_train, augmentation=train_augmentation())
    val_data = Airplane(X_valid, y_valid, augmentation=valid_augmentation())
    BATCH_SIZE = 128
    trainloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)  
    valloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
    
    net = models.resnet18(pretrained=True).to(device)
    fc_inputs = net.fc.in_features

    net.fc = nn.Sequential(
        nn.Linear(fc_inputs, 1)
    ).to(device)
    net.aux_logits=False
    optimizer = optim.Adam(net.parameters(), lr = 0.00075)
    criterion = nn.BCEWithLogitsLoss()
    
    train_losses, val_losses, roc_score = train_model(net, device, criterion, optimizer, trainloader, valloader, 10,n_fold)
    best_epochs.append(f'epoch_{np.array(roc_score).argmax()}_fold_{n_fold}.pth')
    net.load_state_dict(torch.load(f'epoch_{np.array(roc_score).argmax()}_fold_{n_fold}.pth'))
    y_preds += inference_fn(net, inference_dataloader, device).reshape(-1)
    n_fold+=1

# Check

In [ ]:
plt.plot(train_losses, label = 'Train loss')
plt.plot(val_losses, label = 'Val loss')
plt.xlabel('epochs')
plt.ylabel('losses')
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot(roc_score, label='ROC score')
plt.xlabel('epochs')
plt.ylabel('score')
plt.grid()
plt.legend()
plt.show()

In [ ]:
best_epochs